<a href="https://colab.research.google.com/github/JUANES31081/PORTAFOLIO_/blob/main/Red%20PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [ ]:
!mv 'cornell movie-dialogs corpus' 'cornell-movie-dialogs-corpus'

In [ ]:
ls

'cornell movie-dialogs corpus'/     __MACOSX/      text.txt
 cornell_movie_dialogs_corpus.zip   sample_data/   text.zip


In [ ]:
# Crear las secuencias de entrada
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Crear los predictores y las etiquetas
predictors, label = input_sequences[:,:],input_sequences[:,-1]

# Convertir las etiquetas a categóricas
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Cargar el modelo pre-entrenado BERT sin la capa de "pooling"
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Crear el modelo Transformer
input_word_ids = layers.Input(shape=(max_sequence_len,), dtype=tf.int32, name="input_word_ids")
sequence_output = bert_model(input_word_ids)[0]
cls_token = sequence_output[:, 0, :]
out = layers.Dense(total_words, activation='softmax')(cls_token)

model = Model(inputs=input_word_ids, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(predictors, label, epochs=1, verbose=1)

In [ ]:
from transformers import TFDistilBertModel
from tensorflow.keras.callbacks import EarlyStopping

# Crear las secuencias de entrada
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Crear los predictores y las etiquetas
predictors, label = input_sequences[:,:],input_sequences[:,-1]

# Convertir las etiquetas a categóricas
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Cargar el modelo pre-entrenado DistilBERT
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# Crear el modelo Transformer
input_word_ids = layers.Input(shape=(max_sequence_len,), dtype=tf.int32, name="input_word_ids")
sequence_output = distilbert_model(input_word_ids)[0]
cls_token = sequence_output[:, 0, :]
out = layers.Dense(total_words, activation='softmax')(cls_token)

model = Model(inputs=input_word_ids, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# Crear un callback para el early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Entrenar el modelo
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[early_stopping], validation_split=0.3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/100
189/189 [==============================] - 52s 212ms/step - loss: 6.6201 - accuracy: 0.0318 - val_loss: 6.4260 - val_accuracy: 0.0278
Epoch 2/100
189/189 [==============================] - 36s 188ms/step - loss: 6.2764 - accuracy: 0.0348 - val_loss: 6.3524 - val_accuracy: 0.0464
Epoch 3/100
189/189 [==============================] - 38s 201ms/step - loss: 6.2121 - accuracy: 0.0356 - val_loss: 6.3539 - val_accuracy: 0.0464
Epoch 4/100
189/189 [==============================] - 38s 201ms/step - loss: 6.1803 - accuracy: 0.0355 - val_loss: 6.3645 - val_accuracy: 0.0464
Epoch 5/100
189/189 [==============================] - 36s 189ms/step - loss: 6.1599 - accuracy: 0.0331 - val_loss: 6.3617 - val_accuracy: 0.0265


In [ ]:
# Paso 1: Importar las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from transformers import TFBertModel

# Paso 2: Cargar y preprocesar los datos
# Crear un DataFrame vacío
movie_lines = pd.DataFrame(columns=['Line'])

# Leer el archivo línea por línea
with open('/content/cornell-movie-dialogs-corpus/movie_lines.txt', 'r', encoding='ISO-8859-1') as f:
    for i, line in enumerate(f):
        # Limitar el número de líneas a 10000 (o cualquier número que elijas)
        if i >= 1500:
            break
        # Dividir la línea en sus componentes
        components = line.strip().split(' +++$+++ ')
        # Asegurarse de que la línea tenga el número correcto de componentes
        if len(components) == 5:
            # Agregar solo el texto de la línea al DataFrame
            movie_lines = movie_lines.append(pd.Series(components[4], index=['Line']), ignore_index=True)

# Continuar con el preprocesamiento
movie_lines = movie_lines['Line']
# Eliminar líneas vacías o que contienen solo espacios en blanco
movie_lines = movie_lines[movie_lines.str.strip() != '']

# Preprocesar los datos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(movie_lines)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in movie_lines:
    token_list = tokenizer.texts_to_sequences([line])
    if token_list and token_list[0]:  # Asegurarse de que token_list no esté vacío
        token_list = token_list[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(x) for x in input_sequences])

<ipython-input-3-51837d7d0d74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_lines = movie_lines.append(pd.Series(components[4], index=['Line']), ignore_index=True)
<ipython-input-3-51837d7d0d74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_lines = movie_lines.append(pd.Series(components[4], index=['Line']), ignore_index=True)
<ipython-input-3-51837d7d0d74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_lines = movie_lines.append(pd.Series(components[4], index=['Line']), ignore_index=True)
<ipython-input-3-51837d7d0d74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie_lines = movie_lines.append(pd.Se

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from transformers import TFDistilBertModel
from tensorflow.keras.callbacks import EarlyStopping

# Crear las secuencias de entrada
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Crear los predictores y las etiquetas
predictors, label = input_sequences[:,:],input_sequences[:,-1]

# Convertir las etiquetas a categóricas
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


def build_model(hp):
    # Cargar el modelo pre-entrenado DistilBERT
    distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Crear el modelo Transformer
    input_word_ids = layers.Input(shape=(max_sequence_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = distilbert_model(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = layers.Dense(total_words, activation='softmax')(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)

    # Usar un optimizador seleccionado por el ajuste de hiperparámetros
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'SGD', 'RMSprop'])

    model.compile(
        optimizer=hp_optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()

tuner.search(predictors, label, epochs=7, validation_split=0.3)

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
#####
##### CODIGO CON ACTUALIZACIONES
#####

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from transformers import TFDistilBertModel
from tensorflow.keras.callbacks import EarlyStopping

# Crear las secuencias de entrada
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Crear los predictores y las etiquetas
predictors, label = input_sequences[:,:],input_sequences[:,-1]

# Convertir las etiquetas a categóricas
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Define el callback de EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True
)

def build_model(hp):
    # Cargar el modelo pre-entrenado DistilBERT
    distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Crear el modelo Transformer
    input_word_ids = layers.Input(shape=(max_sequence_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = distilbert_model(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = layers.Dense(total_words, activation='softmax')(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)

    # Usar un optimizador seleccionado por el ajuste de hiperparámetros
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Ftrl', 'Nadam'])

    model.compile(
        optimizer=hp_optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']# , 'Precision', 'Recall'] es posible agregar mas metricas de verse necesarias
    )

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search_space_summary()

tuner.search(predictors, label, epochs=10, validation_split=0.3, callbacks=[early_stopping])  # Agrega el callback aquí

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [ ]:
# Continuar el entrenamiento
history = best_model.fit(predictors, label, epochs=100, verbose=1, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/100
189/189 [==============================] - 52s 201ms/step - loss: 3.4450 - accuracy: 0.5089 - val_loss: 3.4066 - val_accuracy: 0.5467
Epoch 2/100
189/189 [==============================] - 36s 189ms/step - loss: 3.0725 - accuracy: 0.5689 - val_loss: 3.3121 - val_accuracy: 0.5461
Epoch 3/100
189/189 [==============================] - 38s 201ms/step - loss: 2.7122 - accuracy: 0.6255 - val_loss: 2.7771 - val_accuracy: 0.6488
Epoch 4/100
189/189 [==============================] - 38s 201ms/step - loss: 2.6303 - accuracy: 0.6394 - val_loss: 2.6972 - val_accuracy: 0.6634
Epoch 5/100
189/189 [==============================] - 38s 201ms/step - loss: 2.1409 - accuracy: 0.7196 - val_loss: 2.4255 - val_accuracy: 0.6998
Epoch 6/100
189/189 [==============================] - 38s 201ms/step - loss: 1.8888 - accuracy: 0.7786 - val_loss: 2.2503 - val_accuracy: 0.7263
Epoch 7/100
189/189 [==============================] - 38s 201ms/step - loss: 1.6682 - accuracy: 0.8311 - val_loss: 2.2177 -

In [ ]:
cat /content/cornell_movie_dialogs_corpus/movie_lines.txt

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # Aplicar la temperatura
        predicted_probs = np.log(predicted_probs) / temperature
        exp_preds = np.exp(predicted_probs)
        predicted_probs = exp_preds / np.sum(exp_preds)
        # Elegir la siguiente palabra
        predicted = np.random.choice(range(len(predicted_probs)), p=predicted_probs)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

In [ ]:
def generate_response(question, model, max_sequence_len):
    # Inicializar la respuesta con la pregunta
    response = question
    # Generar palabras hasta que se genere un signo de interrogación o se alcance un límite de palabras
    for _ in range(10):  # Límite de palabras
        token_list = tokenizer.texts_to_sequences([response])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted = np.argmax(predicted_probs)
        output_word = tokenizer.index_word[predicted] if predicted > 0 else ''
        # Si se genera un signo de interrogación, terminar la respuesta
        if output_word == '?':
            break
        response += " " + output_word
    return response

In [ ]:
def generate_response_r(question, model, max_sequence_len):
    # Inicializar la respuesta con la pregunta
    response = question
    # Generar palabras hasta que se genere un signo de puntuación o se alcance un límite de palabras
    for _ in range(50):  # Límite de palabras
        token_list = tokenizer.texts_to_sequences([response])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted = np.argmax(predicted_probs)
        output_word = tokenizer.index_word[predicted] if predicted > 0 else ''
        # Si se genera un signo de puntuación, terminar la respuesta
        if output_word in ['.', '!', '?']:
            break
        response += " " + output_word
    return response

In [ ]:
# Generar 10 palabras a partir del texto semilla "Hello"
generated_text = generate_text("i am the ", 20, best_model, max_sequence_len,temperature= 0.2)
print(generated_text)

who are are are are are are are are are are are are are are are are are are are are are


In [ ]:
# Define tu pregunta
question = "Who are?"

# Usa la función generate_response para obtener una respuesta del modelo
response = generate_response_r(question, best_model, max_sequence_len)

# Imprime la respuesta
print(response)

Who are? the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
